In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Yashar\\End-to-End-Employee-Classification-with-MLOPs\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Yashar\\End-to-End-Employee-Classification-with-MLOPs'

- In this stage we need URI for MLFlow. 
- we need dagshub as well

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Yashar7800/End-to-End-Employee-Classification-with-MLOPs.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'yashar7800'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '@Lucy1378'

## config_entity.py

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    preprocessor_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

## configuration.py

In [12]:
# defining Confuiguration
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories,save_json


class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.GradientBoostingClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            preprocessor_path= config.preprocessor_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/Yashar7800/End-to-End-Employee-Classification-with-MLOPs.mlflow"
        )

        return model_evaluation_config

## components/model_evaluation.py

In [13]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from pathlib import Path


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred, proba):
        f1 = f1_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        roc_auc = roc_auc_score(actual, proba)
        avg_precision = average_precision_score(actual, proba)

        return f1, precision, recall, roc_auc, avg_precision


    def log_into_mlflow(self):

        data = pd.read_csv(self.config.data_path)
        data['Tenure'] = 2025 - data['JoiningYear']
        X = data.drop(self.config.target_column,axis =1)
        y = data[self.config.target_column]

        model = joblib.load(self.config.model_path) # loading the model

        # Define stratified k-fold
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        # Initialize metrics
        f1_scores, precisions, recalls, auc_roc_scores, auc_pr_scores = [], [], [], [], []


        mlflow.set_registry_uri(self.config.mlflow_uri) # doing everything inside a remote server
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            # Cross-validation
            for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                
                # Fit model on training fold
                model.fit(X_train, y_train)
                
                # Predict on test fold
                y_pred = model.predict(X_test)
                y_proba = model.predict_proba(X_test)[:, 1]
                
                # Compute metrics
                f1, precision, recall, roc_auc, avg_precision = self.eval_metrics(y_test, y_pred, y_proba)
                
                # Store metrics
                f1_scores.append(f1)
                precisions.append(precision)
                recalls.append(recall)
                auc_roc_scores.append(roc_auc)
                auc_pr_scores.append(avg_precision)
                
                # Log per-fold metrics
                mlflow.log_metric(f"f1_score_fold_{fold}", f1)
                mlflow.log_metric(f"precision_fold_{fold}", precision)
                mlflow.log_metric(f"recall_fold_{fold}", recall)
                mlflow.log_metric(f"auc_roc_fold_{fold}", roc_auc)
                mlflow.log_metric(f"auc_pr_fold_{fold}", avg_precision)
                print(f"Fold {fold}: F1={f1:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, AUC-ROC={roc_auc:.4f}, AUC-PR={avg_precision:.4f}")

            # Compute average and std
            avg_f1 = np.mean(f1_scores)
            std_f1 = np.std(f1_scores)
            avg_precision = np.mean(precisions)
            avg_recall = np.mean(recalls)
            avg_auc_roc = np.mean(auc_roc_scores)
            avg_auc_pr = np.mean(auc_pr_scores)

            # Log average metrics
            mlflow.log_metric("avg_f1_score", avg_f1)
            mlflow.log_metric("std_f1_score", std_f1)
            mlflow.log_metric("avg_precision", avg_precision)
            mlflow.log_metric("avg_recall", avg_recall)
            mlflow.log_metric("avg_auc_roc", avg_auc_roc)
            mlflow.log_metric("avg_auc_pr", avg_auc_pr)

            # Save metrics locally using save_json
            scores = {
                "avg_f1_score": avg_f1,
                "std_f1_score": std_f1,
                "avg_precision": avg_precision,
                "avg_recall": avg_recall,
                "avg_auc_roc": avg_auc_roc,
                "avg_auc_pr": avg_auc_pr,
                "f1_scores": f1_scores,
                "precisions": precisions,
                "recalls": recalls,
                "auc_roc_scores": auc_roc_scores,
                "auc_pr_scores": auc_pr_scores
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log model parameters
            mlflow.log_params(self.config.all_params)

            # Log model
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="GradientBoostingClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")

            print(f"Average F1-score: {avg_f1:.4f}, Std: {std_f1:.4f}")
            print(f"Average Precision: {avg_precision:.4f}")
            print(f"Average Recall: {avg_recall:.4f}")
            print(f"Average AUC-ROC: {avg_auc_roc:.4f}")
            print(f"Average AUC-PR: {avg_auc_pr:.4f}")

## pipeline/stage_05_model_evaluation.py

In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    
    raise e

[2025-07-15 14:16:01,797: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-15 14:16:01,812: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-15 14:16:01,815: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-15 14:16:01,817: INFO: common: created directory at: artifacts]
[2025-07-15 14:16:01,818: INFO: common: created directory at: artifacts/model_evaluation]
Fold 1: F1=0.7816, Precision=0.8609, Recall=0.7156, AUC-ROC=0.8757, AUC-PR=0.8502
Fold 2: F1=0.7660, Precision=0.8852, Recall=0.6750, AUC-ROC=0.8688, AUC-PR=0.8483
Fold 3: F1=0.7555, Precision=0.8205, Recall=0.7000, AUC-ROC=0.8645, AUC-PR=0.8517
Fold 4: F1=0.7879, Precision=0.8540, Recall=0.7312, AUC-ROC=0.8763, AUC-PR=0.8590
Fold 5: F1=0.7560, Precision=0.8397, Recall=0.6875, AUC-ROC=0.8680, AUC-PR=0.8461
[2025-07-15 14:16:24,070: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/07/15 14:16:29 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Yashar\AppData\Local\Temp\tmp5s5zamf8\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.4.2', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/07/15 14:16:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'GradientBoostingClassifier' already exists. Creating a new version of this model...
2025/07/15 14:16:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GradientBoostingClassifier, version 2
Created version '2' of model 'GradientBoostingClassifier'.


Average F1-score: 0.7694, Std: 0.0132
Average Precision: 0.8521
Average Recall: 0.7019
Average AUC-ROC: 0.8706
Average AUC-PR: 0.8511
🏃 View run welcoming-steed-591 at: https://dagshub.com/Yashar7800/End-to-End-Employee-Classification-with-MLOPs.mlflow/#/experiments/0/runs/89cfe7fb95f24789b142e83a396f0822
🧪 View experiment at: https://dagshub.com/Yashar7800/End-to-End-Employee-Classification-with-MLOPs.mlflow/#/experiments/0
